# 字符串匹配问题

字符串匹配问题是**在一个给定字符串中搜索特定模式出现的所有位置**，这里的给定字符串使用也称作**文本(Text, T)** ，特定模式被称为**模式(Pattern, P)**。传统的字符串匹配问题中的模式是一个比文本短的字符串，不包含特殊的字符，但1974年Fischer和Paterson将通配符 `*` 引入匹配模式中后，字符串匹配问题变得更加通用也更加复杂。

常见的字符串匹配算法包括：暴力匹配、KMP算法、前缀搜索、后缀搜索、有限状态机等，这些算法有着不同的时间/空间复杂度，以及不同的实现难度，在实际应用中我们需要根据场景选择合适的字符串匹配算法。

# 暴力匹配

暴力匹配也叫朴素字符串匹配算法，它的原理很愣头青，就是从文本中不同的位置开始，沿着一定的方向（👉从前向后或从后向前👈）逐个比较文本和模式中对应位置的字符是否相等。如果发现不相等的字符，则继续下一位置重新匹配，如果匹配到模式尾部，则此次匹配成功。

## 复杂度分析

+ **时间复杂度：** 设本文长度为 $n$，模式长度为 $m$，我们需要对文本中 $n-m+1$ 长度的区域依次进行匹配，每次匹配最多进行 $m$ 次比较， 所以最坏情况下的时间复杂度为 $O((n-m+1) \cdot m)$。
+ **空间复杂度：** 该算法没有预处理过程，也不需要额外的储存空间来缓存字符串，因此空间复杂度为 $O(1)$。

In [ ]:
# 暴力匹配算法实现。
import algviz

def bruteStringMatch(text, pattern):
    if (len(text) < len(pattern)):
        return
    viz = algviz.Visualizer(1)
    tex = viz.createTable(1, len(text), [text], cell_size=25, name='Text', show_index=False)
    pat = viz.createTable(1, len(pattern), [pattern], cell_size=25, name='Pattern', show_index=False)
    for i in range(len(text)-len(pattern)+1):
        for j in range(len(pattern)):
            if tex[0][i+j] == pat[0][j]:
                tex.mark(algviz.colors[0], 0, i+j)
                viz.display()
                if j == len(pattern) - 1:
                    tex.mark(algviz.colors[1], 0, i)
            else:
                viz.display()
                break
        tex.removeMark(algviz.colors[0])
        viz.display()
        
text = 'aaacaaab'
pattern = 'aaab'
bruteStringMatch(text, pattern)

# KMP算法

KMP算法的全称为（ Knuth-Morris-Pratt 算法），是对暴力匹配算法的一种改进。通过观察上面的暴力匹配过程可以发现，每一次从一个新的位置开始进行匹配，我们都会不断的扩展文本和模式的**公共子串**，那么能不能从模式字符串的中间部分开始重新匹配呢？  

观察一个模式字符串 `ababcbc`，如果我们已经匹配了 `abab`，但是在下一个位置失配了，我们需要从模式字符串的首部重新开始呢？其实，可以直接从模式的位置3处的字符开始匹配，因为在得到的这个公共子串 `abab` 中的前缀和后缀中的 `ab` 是匹配的，所以可以直接跳过。对于每一个失配位置可以跳过多少距离，我们可以提前计算好并放到一个表中（**最大匹配数表** Partial-Match-Table）。令人开心的是，这个预处理过程只需要考虑模式字符串，和文本字符串无关。

下面的演示实现了最大匹配数表的构建过程，这是一个递归的过程，【编不下去了。。。。参考知乎。】

## 算法复杂度分析

+ **时间复杂度：** $O(m+n)$，【TODO】涉及到**摊还原分析**。
+ **空间复杂度：** 最大匹配数表的长度等于模式字符串长度，所需储存空间为 $O(m)$。

In [ ]:
# KMP算法实现（递归版本，参考知乎上的回答）。
import algviz

class KMPStringMatch():
    def __init__(self):
        self.viz = algviz.Visualizer(delay=1, wait=True)

    def getPartialMatchTable(self, pat_):
        self.P = len(pat_)
        self.pat = self.viz.createTable(1, self.P, [pat_], cell_size=20, name='Pattern', show_index=False)
        self.pmt = self.viz.createTable(1, self.P, None, cell_size=20, name='MatchTable', show_index=False)
        match, self.pmt[0][0] = 0, 0
        for i in range(1, self.P):
            while match > 0 and self.pat[0][match] != self.pat[0][i]:
                match = self.pmt[0][match - 1]    # 当不匹配的时候，不断回溯。
                self.viz.display()
            if self.pat[0][match] == self.pat[0][i]:
                match += 1                     # 检测是匹配上了还是match到达了模式首部。
            self.pmt[0][i] = match
            self.viz.display()
            
    def match(self, txt_):
        txt = self.viz.createTable(1, len(txt_), [txt_], cell_size=20, name='Text', show_index=False)
        pos = 0
        for i in range(len(txt_)):
            while pos > 0 and txt[0][i] != self.pat[0][pos]:
                pos = self.pmt[0][pos - 1]
                self.viz.display()
            else:
                txt.removeMark(algviz.colors[0])
                for j in range(i-pos, i):
                    txt.mark(algviz.colors[0], 0, j)
            if txt[0][i] == self.pat[0][pos]:
                txt.mark(algviz.colors[0], 0, i)
                pos += 1
            if pos == self.P:
                txt.mark(algviz.colors[1], 0, i-pos+1)
                pos = self.pmt[0][pos-1]
            self.viz.display()
        txt.removeMark(algviz.colors[0])
        self.viz.display()

kmp_solver = KMPStringMatch()
pattern = 'ababaac'
kmp_solver.getPartialMatchTable(pattern)
text = 'abababaacaa'
kmp_solver.match(text)

# 参考链接

+ http://jakeboxer.com/blog/2009/12/13/the-knuth-morris-pratt-algorithm-in-my-own-words/
+ https://www.zhihu.com/question/21923021